In [ ]:
import numpy as np
rel_path = './'

In [ ]:
def clip(input, qbit):
  max_value = 2. ** (qbit-1) -1.
  min_value = -2. ** (qbit-1)
  output = np.clip(input, min_value, max_value)
  return output

def uniform_quantize(input, qbit):
    # 1) 배열화 + NaN/Inf 방어
    x = np.asarray(input, dtype=np.float32)
    x = np.where(np.isfinite(x), x, 0.0)  # NaN/Inf -> 0

    # 2) 대칭 범위 최대값
    A = float(np.max(np.abs(x))) if x.size > 0 else 0.0

    # 3) 스케일 계산(원래 공식 유지), A==0 방어
    qmax_u = float(2**qbit - 1)  # qmin=0, qmax=2^qbit-1 가정
    if A == 0.0 or qbit <= 0:
        # 모두 0이거나 qbit 비정상인 경우: 전부 0으로, scale=1.0
        return np.zeros_like(x, dtype=np.int32), 1.0

    scale = (2.0 * A) / qmax_u  # == (max_cond - min_cond) / (qmax - qmin)

    # 4) 양자화: floor 유지
    q = np.floor(x / scale)

    # 5) signed range로 클립 후 int32 반환
    q = clip(q, qbit).astype(np.int32)
    return q, float(scale)

In [3]:
wdir = rel_path + 'tmpdata/'

fp_w0 = wdir + 'w0.fc1.weight.csv'
fp_w0_scale = wdir + 'w0.fc1.weight_scale.csv'
np_w0 = np.loadtxt(fp_w0, delimiter=',', dtype=float)
np_w0_scale = np.loadtxt(fp_w0_scale, delimiter=',', dtype=float)

fp_w2 = wdir + 'w2.fc2.weight.csv'
fp_w2_scale = wdir + 'w2.fc2.weight_scale.csv'
np_w2 = np.loadtxt(fp_w2, delimiter=',', dtype=float)
np_w2_scale = np.loadtxt(fp_w2_scale, delimiter=',', dtype=float)


In [4]:
# ol?

In [5]:
# PYNQ-Z2 Initialize
from pynq import Overlay, Interrupt
ol = Overlay("./design_top.bit")
top = ol.top_0

def dec_to_tc(data, bit=8):
    if data >= 0:
        return data
    else:
        datatc = pow(2,bit-1) + (pow(2,bit-1)+data)
        return int(datatc)
def tc_to_dec(data, bit=32):
    if data >= pow(2,bit-1):
        datadec = data-2*pow(2,bit-1)
        return int(datadec)
    else:
        return data
def instr_param(opvalid, opcode, param, data):
    _instr = opvalid * pow(2,31)
    _instr += opcode * pow(2,28)
    _instr += param * pow(2,8)
    _instr += data * pow(2,0)
    return int(_instr)    

def instr_data(opvalid, opcode, sel, addr, data):
    _instr = opvalid * pow(2,31)
    _instr += opcode * pow(2,28)
    _instr += sel * pow(2,24)
    _instr += addr * pow(2,8)
#     _instr += data * pow(2,0)
    _instr += dec_to_tc(data)
    return int(_instr)
def pl_rst():
    top.mmio.write(offset=0, data=dec_to_tc(data=-1,bit=32))
    top.mmio.write(offset=0, data=0)
def finish_check():
    mmio_read = top.mmio.read(offset=4)
    read_data = mmio_read / np.power(2,16) / np.power(2,15)
    return int(read_data)
    
# parameters setting
OPCODE_NOP = 0
OPCODE_PARAM = 1
OPCODE_LDSRAM = 2
OPCODE_STSRAM = 3
OPCODE_EX = 4
OPCODE_WBPSRAM = 5
OPCODE_WBPARAM = 6

PARAM_BASE_WSRAM = 0
PARAM_S = 1
PARAM_OC = 2
PARAM_IC = 3
PARAM_TRG = 4

PARAM_IC_WH = 5
PARAM_BASE_WSRAM_WH = 6

TRG_ISRAM = 0
TRG_WSRAM = 1
TRG_PSRAM = 2

In [6]:
# Load np_w0
baseaddr = 0
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_TRG, data=TRG_WSRAM))
for i in range(128):
    for j in range(784):
        x = i % 4
        y = baseaddr + int(np.floor(i/4)) * 784 + j
        top.mmio.write(offset=0, data=instr_data(opvalid=1, opcode=OPCODE_LDSRAM, sel=x, addr=y, data=np_w0[i*784+j]))

In [7]:
# Load np_w2
baseaddr = int(128/4*784)
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_TRG, data=TRG_WSRAM))
for i in range(10):
    for j in range(128):
        x = i % 4
        y = baseaddr + int(np.floor(i/4)) * 128 + j
        top.mmio.write(offset=0, data=instr_data(opvalid=1, opcode=OPCODE_LDSRAM, sel=x, addr=y, data=np_w2[i*128+j]))

In [8]:
# Parameter Test
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_S, data=1))
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_OC, data=128))
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_IC, data=784%256))
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_IC_WH, data=int(784/256)))
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_WBPARAM, param=PARAM_S, data=0))
print(top.mmio.read(offset=12))
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_WBPARAM, param=PARAM_OC, data=0))
print(top.mmio.read(offset=12))
top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_WBPARAM, param=PARAM_IC, data=0))
print(top.mmio.read(offset=12))

1
128
784


In [9]:
wdir = rel_path + 'tmpdata/'
fp_input_csv = wdir + 'mnist_test_small.csv'
np_input_csv = np.loadtxt(fp_input_csv, delimiter=',', dtype=float)

print(np_input_csv.shape)

from pynq import Clocks
Clocks.fclk0_mhz

(10, 785)


25.0

In [ ]:
testsize = 10

for b in range(testsize):
  _label = int(np_input_csv[b][0])
  _image = np_input_csv[b][1:785]

  # fc1
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_OC, data=128))
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_IC, data=784%256))
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_IC_WH, data=int(784/256)))
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_S, data=4))
#   top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_WBPARAM, param=PARAM_S, data=0))
#   print(top.mmio.read(offset=12))
#   top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_WBPARAM, param=PARAM_OC, data=0))
#   print(top.mmio.read(offset=12))
#   top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_WBPARAM, param=PARAM_IC, data=0))
#   print(top.mmio.read(offset=12))

  np_ia0_q, scale_ia0 = uniform_quantize(input=_image, qbit=8)  
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_TRG, data=TRG_ISRAM))
  for i in range(784):
    x = int(i/8)
    y = i % 8
    top.mmio.write(offset=0, data=instr_data(opvalid=1, opcode=OPCODE_LDSRAM, sel=y, addr=x, data=int(np_ia0_q[i])))
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_TRG, data=TRG_PSRAM))
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_PARAM, param=PARAM_BASE_WSRAM, data=0))

  # DEBUG: 베이스 주소 확인
  top.mmio.write(0, instr_param(1, OPCODE_WBPARAM, PARAM_BASE_WSRAM, 0)); lo = top.mmio.read(12)
  top.mmio.write(0, instr_param(1, OPCODE_WBPARAM, PARAM_BASE_WSRAM_WH, 0)); hi = top.mmio.read(12)
  print("FC2 TILE0 BASE =", int(hi)*256 + int(lo))   # 기대값: 25088

  # 실행
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_EX, param=0, data=0))
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_NOP, param=0, data=0))
  np_oa0 = np.zeros([128], dtype=float)

  while not finish_check():
    pass
  for i in range(int(np.ceil(128/4))):
    for j in range(4):
      if j+4*i < 128:
        top.mmio.write(offset=0, data=instr_data(opvalid=1, opcode=OPCODE_WBPSRAM, sel=j, addr=i, data=0))
        np_oa0[j+4*i]=tc_to_dec(data=top.mmio.read(offset=12),bit=32)
        np_oa0[j+4*i]*= scale_ia0 * np_w0_scale
  top.mmio.write(offset=0, data=instr_param(opvalid=1, opcode=OPCODE_NOP, param=0, data=0))


  #   #fc2
  top.mmio.write(offset=0, data=instr_param(1, OPCODE_PARAM, PARAM_IC,    128))
  top.mmio.write(offset=0, data=instr_param(1, OPCODE_PARAM, PARAM_IC_WH, 0))
  # ReLU
  np_ia2 = np.maximum(np_oa0, 0.0)
  np_ia2_q, scale_ia2 = uniform_quantize(input=np_ia2, qbit=8)

  # 입력 ISRAM 적재
  top.mmio.write(offset=0, data=instr_param(1, OPCODE_PARAM, PARAM_TRG, TRG_ISRAM))
  for i in range(128):
    x = i // 8
    y = i % 8
    top.mmio.write(offset=0, data=instr_data(1, OPCODE_LDSRAM, sel=y, addr=x, data=int(np_ia2_q[i])))

  # 4-pack에서 FC2 베이스 (가중치 로딩과 동일해야 함)
  base_fc2 = int(128/4*784)          # = 25088
  base_fc2_wh = base_fc2 // 256
  base_fc2_tile1 = base_fc2 + 2*128   # = base_fc2 + 256
  base_fc2_tile1_wh = base_fc2_tile1 // 256

  # -------------------------
  # 타일0: S=4, OC=8, 베이스 = base_fc2
  # -------------------------
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_S, 4))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_OC, 8))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_BASE_WSRAM,    base_fc2))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_BASE_WSRAM_WH, base_fc2_wh))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_TRG, TRG_PSRAM))
  top.mmio.write(0, instr_param(1, OPCODE_EX, 0, 0))
  top.mmio.write(0, instr_param(1, OPCODE_NOP, 0, 0))
  while not finish_check(): pass

  # -------------------------
  # 타일1: S=2, OC=2, 베이스 = base_fc2 + 256
  # -------------------------
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_S, 2))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_OC, 2))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_BASE_WSRAM,    base_fc2_tile1))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_BASE_WSRAM_WH, base_fc2_tile1_wh))
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_TRG, TRG_PSRAM))

  # DEBUG: 베이스 주소 확인
  top.mmio.write(0, instr_param(1, OPCODE_WBPARAM, PARAM_BASE_WSRAM, 0)); lo = top.mmio.read(12)
  top.mmio.write(0, instr_param(1, OPCODE_WBPARAM, PARAM_BASE_WSRAM_WH, 0)); hi = top.mmio.read(12)
  print("FC2 TILE1 BASE =", int(hi)*256 + int(lo))   # 기대값: 25344

  # 실행
  top.mmio.write(0, instr_param(1, OPCODE_EX, 0, 0))
  top.mmio.write(0, instr_param(1, OPCODE_NOP, 0, 0))
  while not finish_check(): pass
  
  # -------------------------
  # 읽기: 타일0(0..7)을 S=4 레이아웃으로, 타일1(8,9)을 S=2 레이아웃으로
  # -------------------------
  np_oa2 = np.zeros([10], dtype=float)

  # 타일0 읽기 (S=4): oc 0..7 => addr=oc//4, sel=oc%4
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_S, 4))
  for oc in range(8):
    addr = oc // 4
    sel  = oc % 4
    top.mmio.write(0, instr_data(1, OPCODE_WBPSRAM, sel=sel, addr=addr, data=0))
    val = tc_to_dec(top.mmio.read(12), bit=32)
    np_oa2[oc] = val * scale_ia2 * np_w2_scale

  # 타일1 읽기 (S=2): oc 8,9 => addr=0, sel=0/1
  top.mmio.write(0, instr_param(1, OPCODE_PARAM, PARAM_S, 2))
  for t in range(2):
    sel, addr = t, 0
    top.mmio.write(0, instr_data(1, OPCODE_WBPSRAM, sel=sel, addr=addr, data=0))
    val = tc_to_dec(top.mmio.read(12), bit=32)
    np_oa2[8+t] = val * scale_ia2 * np_w2_scale

  top.mmio.write(0, instr_param(1, OPCODE_NOP, 0, 0))
  _infer = np.argmax(np_oa2)
  print("inference:", _infer, ", answer:", _label)


inference: 7, answer: 7
inference: 2, answer: 2
inference: 1, answer: 1
inference: 0, answer: 0
inference: 4, answer: 4
inference: 1, answer: 1
inference: 4, answer: 4
inference: 9, answer: 9
inference: 6, answer: 5
inference: 9, answer: 9
